<a href="https://colab.research.google.com/github/tluxxx/PortfolioExperiments/blob/main/GebertIndicatorTest(1993_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gebert Indicator

#1. General Preparations

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# import data processing libraries
import pandas as pd
import numpy as np
import datetime as dt

# import plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# import auxilliary labraries
import datetime as dt

# import data providing libraries
import yfinance as yf
import pandas_datareader as pdr

#2. Downloading required data and data preparations

## 2.1 DAX-data

In [231]:
# importing the DAX-data
start_period = '1992-01-01'
start_assessment = '1993-01-01'
end_period = '2024-05-31'
ticker ='^GDAXI'
dax_d = yf.download(ticker, start=start_period, end=end_date)
dax_d['date'] = pd.to_datetime(dax_d.index).date
trade_dates_m = dax_d.resample('MS').first()['date'].to_list()
dax_m = dax_d[dax_d.index.isin(trade_dates_m)]

[*********************100%%**********************]  1 of 1 completed


## 2.2 Inflation Data

In [117]:
# reading and preparing inflation data for 1991-2024
# data from German Statistical Office (1991-1998)
data_ger = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/GermanInflationData(1991-2024).xlsx', sheet_name='DATA')
inflation_ger = pd.Series(data_ger['chg_12m'].tolist(), index=data_ger['date'])

# EU-data from EuroStat (1999-2024)
data_eu = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/EUInflationData(1997-2024).xlsx',sheet_name='DATA')
inflation_eu = pd.Series(data_eu['EU_infl_annual_roc'].tolist(), index=data_eu['date'])

inflation_ger = inflation_ger.loc[start_period:]
inflation_ger.loc[start_period] = np.nan
cut_off_date ='1998-01-31'
df1 = inflation_ger.loc[start_period:cut_off_date]
df2 = inflation_eu.loc[cut_off_date:]
inflation = pd.concat([df1,df2])
inflation.name = 'inflation'


In [118]:
# visualisation of the constituents of the inflation data
main_title = 'Inflation rates Germany and EU '
sub_title = f'from   {start_date_assessment}   to   {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=inflation_ger.index, y=inflation_ger, name='Inflation Germany (German Statistical Office)'))
fig.add_trace(go.Scatter(x=inflation_eu.index, y=inflation_eu, name='Inflation EU (Euro-Stat)'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='inflation rate (%)', legend_title='Sources')
fig.show()

In [119]:
main_title = 'aggregated inflation rates Germany/EU '
sub_title = f'Germany from {start_date_assessment} to {cut_off_date},  EU from {cut_off_date} to {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=inflation.index, y=inflation, name='Inflation Germany & EU'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='inflation-rate(%)')
fig.show()

## 2.3. Interest Rates

In [120]:
# reading and preparing interest data from Central Bank Sources

# German Bundesbank discount rates for every day (approximation, all values as of 01-of each month)
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/DiscountRatesGermanBundesbank (1990-1998).xlsx',sheet_name='DATA' )
data_db['date'] = pd.to_datetime(data_db['date'])

# generation of data frame with relevant data
## monthly data
end_date_int = '1998-12-31'

disc_x = (data_db[data_db['date']>=start_period]['interest'] /100).tolist()
disc_r = pd.DataFrame({'date':data_db[data_db['date']>=start_period]['date'].tolist(),
                       'interest':disc_x})
## conversion to daily data (approximation: if interest rate changes during a month, the change will be implemented at first day of following month
dates = pd.date_range(start_period, end_date_int)
values = np.nan * len(dates)
dates_d = pd.DataFrame({'date': dates,
                        'interest': values})
xx = pd.merge(dates_d, disc_r, how='left', on='date')['interest_y']
xx.ffill(inplace=True)
db_int = pd.DataFrame({'interest':xx.tolist()},
                      index = dates)

# ECB interest data for every day
data_ecb = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/EZB interest rates (1999-2024).xlsx')

ecb_int = pd.DataFrame({'interest': data_ecb['EZB_int'].tolist()},
                       index=data_ecb['DATE'].tolist())

# combination of DB and ECB datasets, results transfered to a pd.Series()
interest = pd.concat([db_int, ecb_int])
interest = interest['interest'].astype(float)


In [121]:
# visualisation of the constituents of the intrest rate
main_title = 'Central Bank interest rates Germany and EU '
sub_title = f'from   {start_date_int}   to   {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=db_int.index, y=db_int['interest']*100, name='Discount Rate German Bundesbank (until 1998-12)'))
fig.add_trace(go.Scatter(x=ecb_int.index, y=ecb_int['interest']*100, name='ECB main financing facility (from 1999-01)'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='interest rate (%)', legend_title='Sources')
fig.show()

In [122]:
main_title = 'aggregated Interest-rates '
sub_title = f'Bundesbank from {start_date_int} to {end_date_int},  ECB from {end_date_int} to {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=interest.index, y=interest, name='Discount Rates Bundesbank'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='intrest-rate (%)')
fig.show()

## 2.3 Exchange rates EURO - USD

In [123]:
# importing the USD/DM historical exchange rates
data_db = pd.read_excel('/content/gdrive/My Drive/ColabNotebooks/VariousTopics/PortfolioTests/data/USDDM_HistoricalRatesAdaptedVersion_1960_1998.xlsx', index_col='Date')

data_db['rate'] = data_db['rate'].str.replace('.', '0')
data_db['rate'] = data_db['rate'].str.replace(',', '.')
data_db['rate'] = data_db['rate'].astype(float)
data_db['rate'].replace(0, np.nan, inplace=True)
data_db['rate'].ffill(inplace=True)

usdeur_db = data_db.loc[start_period:]
usdeur_db = usdeur_db.div(1.95583)
usdeur_db.index.names = ['DATE']
usdeur_db.index = pd.to_datetime(usdeur_db.index)

# importing the USD/EURO historical exchange rates
start_date_x = '1999-01-01'
usdeur_fred = pdr.data.DataReader('DEXUSEU', 'fred', start_date_x, end_date)
usdeur_fred['DEXUSEU'].ffill(inplace=True)
usdeur_fred['rate'] = 1 / usdeur_fred['DEXUSEU']
usdeur_fred.drop(columns=['DEXUSEU'], inplace=True)

# combining Bundesbank and FRED data to one timeseries
df1 = usdeur_db
df2 = usdeur_fred
usdeur = pd.concat([df1,df2])
usdeur = usdeur['rate']
usdeur.name = 'usdeur'

In [124]:
# historical values of exchange rates
main_title = 'Exchange rate EUR-USD and DM-USD (DM converted in EURO)'
sub_title = f'from   {start_date_x}   to   {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=usdeur_db.index, y=usdeur_db['rate'], name='USD/EURO (EURO = 1.9558 DM) (Bundesbank)'))
fig.add_trace(go.Scatter(x=usdeur_fred.index, y=usdeur_fred['rate'], name='USD/EURO (FED)'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='equity', legend_title='Sources')
fig.show()

In [125]:
# plot of the combined timeseries
main_title = 'aggregated exchange rates EURO-USD '
sub_title = f'Bundesbank from {start_date_int}  to  {end_date_int} and FED from {start_date_x} to {end_date} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'
fig = go.Figure()
fig.add_trace(go.Scatter(x=usdeur.index, y=usdeur, name='USD/EUR'))
fig.update_layout(template='plotly_dark', autosize=False, width=1000, height=600)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='equity', legend_title='Sources')
fig.show()

# Generating the indicator

In [237]:
# generation dataframe containing raw data

# generation of list of all first day of the mont and parameter dataframe
dates_param = dates = pd.date_range(start_period, end_period, freq ='MS')
params = pd.DataFrame(index=dates_param)

# collecting first trading dazs of an month
params['trade_days'] = trade_dates_m

# collection of dax-data
x = dax_m.Close.tolist()
params['dax'] = x

# collection of interest data and adding to dataframe
intr = interest[interest.index.isin(dates_param)] * 100
params = params.join(intr)

# collection of inflation data and adding to dataframe
infl = inflation[inflation.index.isin(dates_param)]
params = params.join(infl)

params['inflation_12']=params['inflation'].shift(12)  # Check only

# collection of exchange rate data
currency = usdeur[usdeur.index.isin(trade_dates_m)]
params['eurusd'] = currency.tolist()

params['eurusd_12'] = params['eurusd'].shift(12) # Check only

# collection input for saisonality
params['seasonality'] = params.index.month


In [239]:
# calculation of subindicators, Gebert-Indicator and positions

# subindicator interest: +1 if last adaptation of CB's benchmark interest rate was a reduction, 0 otherwise
interest_chg = params['interest'].diff()
params['ind_int_chg'] = np.select([interest_chg < 0, interest_chg > 0, interest_chg == 0], [1, 0, np.nan])
params['ind_int_chg'].ffill(inplace=True)

# subindicator inflation: +1: if inflation rate declines compared to -12m value, 0 otherwise
params['ind_infl_chg'] = np.where(params['inflation'] < params['inflation_12'], 1, 0)

# subindicator exchange rates USD/EUR:
params['ind_eurusd_chg'] = np.where(params['eurusd'] > params['eurusd'].shift(12), 1, 0)

# subindicator seasonality: +1 during [November -April], 0 otherwise
params['ind_seasonality'] = np.where(params['seasonality'].isin([11,12,1,2,3,4]), 1, 0)

# Gebert-Indicator and Trade postions
params['GI'] = params['ind_int_chg'] + params['ind_infl_chg'] + params['ind_eurusd_chg'] + params['ind_seasonality']

conditions = [params['GI'] <= 1, params['GI'] >= 3, params['GI'] == 2]
choices = [0, 1, np.nan]
params['positions'] = np.select(conditions, choices)
params['positions'].ffill(inplace=True)


In [245]:
# profitability Gebert indicator system - accumulated returns
rets = params['dax'].pct_change() * params['positions']
params['pnl'] = (1 + rets).cumprod()

# profitability Gebert indicator system - constant invest
params['pnl_constant_invest'] = 1 + rets.cumsum()

# Buy and Hold data as reference
rets_buh = params['dax'].loc[start_assessment:].pct_change()
params['pnl_buh'] = (1 + rets_buh).cumprod()


In [246]:
# visualisation of PnL
main_title = 'development of PnL position according to Gebert Indicator vs BuH'
sub_title = f'instrument {ticker} from   {start_assessment}   to   {end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'


fig = make_subplots(rows=2, cols=1, row_heights=[0.9,0.1])
fig.add_trace(go.Scatter(x=params.index, y=params['pnl'], name = 'Gebert Strategy - accumulated returns'), row=1, col=1)
fig.add_trace(go.Scatter(x=params.index, y=params['pnl_constant_invest'], name = 'Gebert Strategy - constant invest'), row=1, col=1)

fig.add_trace(go.Scatter(x=params.index, y=params['pnl_buh'], name = 'Buy and Hold'), row=1, col=1)
fig.add_trace(go.Scatter(x=params.index, y=params['positions'], line_shape='hv', line_color='green', fill='tozeroy', name='position'), row=2, col=1)
fig.update_layout(template='plotly_dark', autosize=False, width=1500, height=900)
fig.update_layout(title=title, xaxis_title='time', yaxis_title='position' )
fig.show()

In [235]:
# Drawdown

x0 = ((params['pnl'] / params['pnl'].expanding().max()) - 1) * 100
x1 = ((params['pnl_buh'] / params['pnl_buh'].expanding().max()) - 1) * 100

main_title = 'drawdown Gebert Strategy vs. Buy & Hold'
sub_title = f'instrument {ticker}   from   {start_assessment}   to   {end_period} '
title = main_title + '<br><br><sup>' + sub_title + '</sup>'

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0.index, y=x0, mode='none', fill='tozeroy', fillcolor='rgba(0, 255, 0, 0.3)', name='Gebert-Strategy'))
fig.add_trace(go.Scatter(x=x1.index, y=x1, mode='none', fill='tozeroy', fillcolor='rgba(255, 255, 0, 0.3)', name='Buy & Hold'))
fig.update_layout(template = 'plotly_dark',autosize=False, width=1500, height=400)
fig.update_layout(title=title, xaxis_title='date', yaxis_title='drawdown (%)')
fig.show()



In [240]:
params.tail(20)

,trade_days,dax,interest,inflation,inflation_12,eurusd,eurusd_12,seasonality,ind_int_chg,ind_infl_chg,ind_eurusd_chg,ind_seasonality,GI,positions
2022-10-01,2022-10-03,12209.480469,1.25,10.9,3.6,1.019472,0.862218,10,0.0,0,1,0,1.0,0.0
2022-11-01,2022-11-01,13338.740234,1.25,11.5,4.4,1.013069,0.862738,11,0.0,0,1,1,2.0,0.0
2022-12-01,2022-12-01,14490.299805,2.00,11.1,5.2,0.952562,0.883158,12,0.0,0,1,1,2.0,0.0
2023-01-01,2023-01-02,14069.259766,2.50,10.4,5.3,0.934754,0.885740,1,0.0,0,1,1,2.0,0.0
2023-02-01,2023-02-01,15180.740234,2.50,10.0,5.6,0.916003,0.889759,2,0.0,0,1,1,2.0,0.0
2023-03-01,2023-03-01,15305.019531,3.00,9.9,6.2,0.936856,0.900657,3,0.0,0,1,1,2.0,0.0
2023-04-01,2023-04-03,15580.919922,3.50,8.3,7.8,0.918189,0.905551,4,0.0,0,1,1,2.0,0.0
2023-05-01,2023-05-02,15726.940430,3.50,8.1,8.1,0.909339,0.950480,5,0.0,0,0,0,0.0,0.0
2023-06-01,2023-06-01,15853.660156,3.75,7.1,8.8,0.930060,0.939320,6,0.0,1,0,0,1.0,0.0
2023-07-01,2023-07-03,16081.040039,4.00,6.4,9.6,0.915835,0.960707,7,0.0,1,0,0,1.0,0.0
